selenium - biblioteka do scrapowania danych
re - biblioteka do analizy i modyfikowania ciągów znaków

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re

Silnik chromium do uruchomienia okna scrapowania strony

In [23]:
driver = webdriver.Chrome()
driver.get("https://krecik.sklep.pl")
driver.set_window_size(1920, 1080)

Scrapowanie odnośników do każdej oferty zestawów klocków LEGO
Selenium przechodzi po każdej stronie a następnie dodaje do listy linki pobrane z każdego kafelka ofert znajdujących się na danym page'u

In [24]:
products = []
for i in range(1,12):
    try:
        driver.get("https://krecik.sklep.pl/pl/c/LEGO-rok-produkcji/51/"+str(i))
        for j in range(1,51):
            products.append(driver.find_element(By.XPATH, "/html/body/div[2]/div[5]/div/div/div[2]/div/div[2]/div[1]/div["+str(j)+"]/div/a[2]").get_attribute("href"))
    except:
        break

Z powodu wielu różnych opcji formatowania i zapisu informacji o zestawach, używamy regexa w celu ujednolicenia zapisu danych oraz podziału jednego paragrafu na osobne zmienne

In [25]:
def parse_output_type_1(output_str):
    match = re.search(r'Numer zestawu: (\d+)', output_str)
    id = int(match.group(1)) if match else None

    match = re.search(r'Seria glowna: (.+)', output_str)
    series = match.group(1) if match else None

    match = re.search(r'Rok produkcji: (\d+)', output_str)
    year = int(match.group(1)) if match else None

    match = re.search(r'Liczba elementów: (\d+)', output_str)
    elements = int(match.group(1)) if match else None

    match = re.search(r'Liczba figurek: (\d+)', output_str)
    figures = int(match.group(1)) if match else None

    match = re.search(r'instrukcja: (\w+)', output_str)
    manual = True if match and match.group(1).lower() == 'tak' else False

    match = re.search(r'pudełko: (\w+)', output_str)
    box = True if match and match.group(1).lower() == 'tak' else False

    return id, series, year, elements, figures, manual, box


def parse_output_type_2(output_str):
    match = re.search(r'Numer zestawu (\d+)', output_str)
    id = int(match.group(1)) if match else None

    match = re.search(r'Seria główna (.+)', output_str)
    series = match.group(1) if match else None

    match = re.search(r'Rok (\d+)', output_str)
    year = int(match.group(1)) if match else None

    match = re.search(r'Liczba elementów (\d+)', output_str)
    elements = int(match.group(1)) if match else None

    match = re.search(r'Liczba minifigurek (\d+)', output_str)
    figures = int(match.group(1)) if match else None

    match = re.search(r'Czy jest pudełko\? (\w+)', output_str)
    box = True if match and match.group(1).lower() == 'tak' else False

    match = re.search(r'Czy jest instrukcja\? (\w+)', output_str)
    manual = True if match and match.group(1).lower() == 'tak' else False

    return id, series, year, elements, figures, manual, box

def parse_output(output_str):
    if re.search(r'Numer zestawu:', output_str) and re.search(r'Nazwa zestawu:', output_str):
        return parse_output_type_1(output_str)
    
    elif re.search(r'Numer zestawu', output_str) and re.search(r'Seria główna', output_str):
        return parse_output_type_2(output_str)
    else:
        return None, None, None, None, None, None, None

Selenium przechodzi po każdej zapisanej ofercie i scrapuje ich ceny oraz specyfikacje
Na koniec wszystko pakowane jest w dataframe i eksportowane do excela

In [26]:
products_info = [('price', 'id', 'series', 'year', 'element', 'figures', 'manual', 'box')]
for product in products:
    driver.get(product)
    price = driver.find_element(By.XPATH, "/html/body/div[2]/div[5]/div/div/div[2]/div[1]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/em").text
    try:
        specs = driver.find_element(By.XPATH, "/html/body/div[2]/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/table/tbody").text
    except:
        specs = driver.find_element(By.XPATH, "/html/body/div[2]/div[5]/div/div/div[2]/div[2]/div[1]/div[2]/div").text
    id, series, year, elements, figures, manual, box = parse_output(specs)
    products_info.append(tuple((price, id, series, year, elements, figures, manual, box)))
pd.DataFrame(products_info).to_excel('scraped_products.xlsx', header=False, index=False)